# IT1244 Project

## Import Libraries

In [1]:
import re as re
import heapq as heapq
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MultiLabelBinarizer, Normalizer
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
import torch as torch

In [3]:
from transformers import BertModel
from transformers import BertTokenizer, BertTokenizerFast

model = BertModel.from_pretrained('bert-base-uncased')

c:\Users\irvin\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\irvin\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\irvin\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an adminis

In [4]:

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Encode sentence
sentence = "This is an example sentence."
inputs = tokenizer(sentence, return_tensors="pt")

# Get embeddings
with torch.no_grad():
    outputs = model(**inputs)

# Extract [CLS] token embedding (sentence-level representation)
cls_embedding = outputs.last_hidden_state[:, 0, :]
print(cls_embedding.shape)  # Should be (1, 768)

torch.Size([1, 768])


In [5]:
cls_embedding

tensor([[-3.7736e-01, -3.3498e-01, -3.2064e-01, -4.3991e-01, -5.0140e-01,
         -2.1692e-01,  4.7474e-01,  5.7114e-01, -8.4856e-02, -1.0667e-01,
         -3.7948e-01, -4.8466e-03, -9.6260e-02,  1.4302e-02,  6.4120e-01,
          4.2318e-02,  2.1465e-01,  5.6647e-01,  2.4406e-01, -4.9750e-01,
          2.4432e-01, -3.1471e-01, -1.8387e-01, -4.3433e-01,  3.7000e-01,
         -4.8847e-01, -1.6026e-01, -5.3024e-01, -2.5536e-01,  2.5280e-02,
          1.1406e-01,  3.2830e-01, -3.3749e-01, -1.7455e-01,  5.4211e-01,
         -2.4419e-01,  2.1843e-01, -5.2654e-02,  5.4251e-01,  2.2136e-01,
         -2.6010e-01, -6.9896e-02,  4.5415e-01,  1.4498e-01,  7.1129e-02,
         -1.9316e-01, -2.8830e+00, -4.7826e-01, -4.7273e-01, -2.8777e-01,
         -3.7046e-01, -8.1553e-02,  7.4794e-01,  4.6832e-01, -2.2192e-01,
          3.8314e-01, -3.8344e-01,  3.7906e-01,  2.1074e-01,  1.0201e-02,
          3.5558e-01,  4.1932e-02, -1.1501e-02, -9.0390e-02, -2.5045e-02,
          6.6402e-01, -3.8978e-02,  5.

## Data Importing

In [46]:
tweets = pd.read_csv("../Data/Raw/Tweets.csv")
tweets.head()

,airline_sentiment,sentiment_confidence,text
0,neutral,1.0000,@VirginAmerica What @dhepburn said.
1,positive,0.3486,@VirginAmerica plus you've added commercials t...
2,neutral,0.6837,@VirginAmerica I didn't today... Must mean I n...
3,negative,1.0000,@VirginAmerica it's really aggressive to blast...
4,negative,1.0000,@VirginAmerica and it's a really big bad thing...


### Code starts here

In [47]:
tweets.shape


(14639, 3)

In [48]:
tweets["text"][1]

"@VirginAmerica plus you've added commercials to the experience... tacky."

In [49]:
def at_filter(text):
    return re.sub(r"@\w+", "", text)

In [50]:
def alpha_filter(text):
    return re.sub(r'[^A-Za-z ]', '', text)

## Data Cleaning

In [51]:
tweets["text"] = tweets["text"].apply(lambda x: at_filter(x))


In [52]:
tweets["text"] = tweets["text"].apply(lambda x: alpha_filter(x))

In [53]:
tweets

,airline_sentiment,sentiment_confidence,text
0,neutral,1.0000,What said
1,positive,0.3486,plus youve added commercials to the experienc...
2,neutral,0.6837,I didnt today Must mean I need to take anothe...
3,negative,1.0000,its really aggressive to blast obnoxious ente...
4,negative,1.0000,and its a really big bad thing about it
...,...,...,...
14634,positive,0.3487,thank you we got on a different flight to Chi...
14635,negative,1.0000,leaving over minutes Late Flight No warnings...
14636,neutral,1.0000,Please bring American Airlines to BlackBerry
14637,negative,1.0000,you have my money you change my flight and do...


In [54]:
tweets.to_csv("../Data/Cleaned/CleanTweets.csv")

In [57]:
max(list(map(lambda x: len(x), tweets["text"])))

166